In [ ]:
# Função para reconhecimento facial e análise de emoções
def analyze_faces(frame, front_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    front_faces = front_cascade.detectMultiScale(gray, 1.3, 5)
    emotions = []
    
    faces = []
    faces.extend(front_faces)  
    # faces.extend(profile_faces)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        
        # Análise de emoções
        analysis = DeepFace.analyze(roi, actions=['emotion'], enforce_detection=False)
        emotions.append(analysis[0]['dominant_emotion'])
        
        # Marcação da face no vídeo
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, analysis[0]['dominant_emotion'], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, faces, emotions

# Função para detecção de anomalias nos movimentos (base simples usando diferença de frames)
def detect_anomalies(prev_frame, curr_frame):
    frame_diff = cv2.absdiff(curr_frame, prev_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_diff, 25, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    anomalies = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Área mínima para considerar como anômalo
            anomalies.append(contour)
    
    return anomalies

# Função principal para análise de vídeo
def process_video(input_video_path, output_video_path):
    # Inicializando o detector de rostos (Haar Cascade)
    front_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
    
    # Abrindo o vídeo
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Configurando o vídeo de saída
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))
    
    # Variáveis para análise
    total_frames = 0
    anomalies_detected = 0
    prev_frame = None
    emotions_data = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        total_frames += 1
        frame_copy = frame.copy()
        
        # 1. Reconhecimento facial e análise de emoções
        frame, faces, emotions = analyze_faces(frame, front_cascade)
    
        # 2. Detecção de anomalias
        if prev_frame is not None:
            anomalies = detect_anomalies(prev_frame, frame)
            anomalies_detected += len(anomalies)
            
            for anomaly in anomalies:
                cv2.drawContours(frame, [anomaly], -1, (0, 0, 255), 2)
        
        # Armazenando os dados das emoções
        emotions_data.extend(emotions)
        
        # Salvando o frame processado no vídeo de saída
        out.write(frame)
        
        prev_frame = frame_copy
    
    # Finalizando o vídeo
    cap.release()
    out.release()
    
    # Gerar relatório
    report = {
        "total_frames": total_frames,
        "anomalies_detected": anomalies_detected,
        "emotions_data": emotions_data
    }
    
    # Salvar o relatório em JSON
    with open('video_analysis_report.json', 'w') as f:
        json.dump(report, f, indent=4)

    print("Processamento completo. Relatório gerado e vídeo de saída salvo.")
    return report

# Execução do código
input_video_path = 'video_analise.mp4'  # Caminho para o vídeo de entrada
output_video_path = 'output_video.avi'  # Caminho para o vídeo de saída

report = process_video(input_video_path, output_video_path)

# Exibir o relatório
print("Relatório:")
print(f"Total de frames analisados: {report['total_frames']}")
print(f"Número de anomalias detectadas: {report['anomalies_detected']}")
print(f"Emoções detectadas: {report['emotions_data']}")